# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846861271543                   -0.70    4.5         
  2   -7.852319760086       -2.26       -1.53    1.0   17.0ms
  3   -7.852615245090       -3.53       -2.56    1.5   18.5ms
  4   -7.852646007991       -4.51       -2.88    2.5   71.3ms
  5   -7.852646508230       -6.30       -3.18    1.0   16.7ms
  6   -7.852646680876       -6.76       -4.13    1.0   17.1ms
  7   -7.852646686652       -8.24       -5.15    1.8   20.9ms
  8   -7.852646686725      -10.13       -5.50    1.8   20.2ms
  9   -7.852646686728      -11.53       -5.68    1.0   17.5ms
 10   -7.852646686730      -11.80       -6.76    1.0   17.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846937106877                   -0.70           4.5         
  2   -7.852555930781       -2.25       -1.64   0.80    2.2    195ms
  3   -7.852639517983       -4.08       -2.74   0.80    1.0   16.0ms
  4   -7.852646442035       -5.16       -3.35   0.80    1.8   18.6ms
  5   -7.852646679870       -6.62       -4.24   0.80    2.2   20.1ms
  6   -7.852646686503       -8.18       -4.78   0.80    2.0   18.8ms
  7   -7.852646686724       -9.66       -5.63   0.80    1.5   59.0ms
  8   -7.852646686730      -11.23       -6.76   0.80    2.0   19.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.442963e+01     3.744036e+00
 * time: 0.10014796257019043
     1     1.386925e+00     1.843700e+00
 * time: 0.2678661346435547
     2    -1.644264e+00     1.890967e+00
 * time: 0.28575897216796875
     3    -3.793762e+00     1.669979e+00
 * time: 0.31145501136779785
     4    -5.293099e+00     1.391467e+00
 * time: 0.33708715438842773
     5    -6.891610e+00     8.160903e-01
 * time: 0.3617730140686035
     6    -7.070990e+00     1.003226e+00
 * time: 0.37882018089294434
     7    -7.594802e+00     6.581835e-01
 * time: 0.39580512046813965
     8    -7.692329e+00     5.628682e-01
 * time: 0.4133939743041992
     9    -7.788861e+00     2.160253e-01
 * time: 0.4311549663543701
    10    -7.822861e+00     8.019360e-02
 * time: 0.44838809967041016
    11    -7.838386e+00     1.057584e-01
 * time: 0.46537208557128906
    12    -7.847317e+00     5.566513e-02
 * time: 0.4823460578918457
    13    -7.850747e+00     2.883442e-02
 * time: 0.50

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846863056849                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645897102                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.50s
  3   -7.852646686730      -13.27       -7.23    121ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1829138156774227e-7
|ρ_newton - ρ_scfv| = 2.2928493120437697e-7
|ρ_newton - ρ_dm|   = 1.3805075314868522e-9
